# Domain adaptation on classification task with AlexNet and PACS dataset

In [2]:
# Install additional libraries
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/SimoneBorella/domain-adaptation.git

!cp -r ./domain-adaptation/models .
!cp -r ./domain-adaptation/utils .

!rm -r ./domain-adaptation
!rm -r sample_data

In [ ]:
# Download PACS Dataset Images
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!mkdir -p data
!mv Homework3-PACS/PACS data/
!rm -r Homework3-PACS/

# Download PACS Dataset Labels
!git clone https://github.com/silvia1993/DANN_Template/
!mv DANN_Template/txt_lists/art_painting.txt data/PACS/
!mv DANN_Template/txt_lists/cartoon.txt data/PACS/
!mv DANN_Template/txt_lists/photo.txt data/PACS/
!mv DANN_Template/txt_lists/sketch.txt data/PACS/
!rm -r DANN_Template/

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032 (from 1)
Receiving objects: 100% (10032/10032), 174.13 MiB | 8.85 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
rm: remove write-protected regular file 'Homework3-PACS/.git/objects/pack/pack-593e21a0699e0596d1434535007769eaee8f5f32.idx'? ^C
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23 (from 1)
Receiving objects: 100% (23/23), 33.86 KiB | 990.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.
rm: remove write-protected regular file 'DANN_Template/.git/objects/pack/pack-e4becacbe1d106caa2e769f6634d14d1068f1ee3.pack'? 

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from PIL import Image

from torch.utils.data import DataLoader
from torch.backends import cudnn
import torchvision.transforms as T
from torchvision.models import AlexNet_Weights
import torch.nn.functional as F
from torchmetrics import Accuracy

import matplotlib.pyplot as plt

from models.alexnet import AlexNet
from utils.monitor import Monitor

In [ ]:
# Google colab
from google.colab import drive
drive.mount('/content/drive')
! mkdir -p /content/drive/MyDrive/da
res_dir = "/content/drive/MyDrive/da"


# Local
# res_dir = "."

In [4]:
def get_device():
    if torch.cuda.is_available():
        print("CUDA available")
        print(f"Number of devices: {torch.cuda.device_count()}")
        for dev in range(torch.cuda.device_count()):
            print(f"Device {dev}:")
            print(f"\tName: {torch.cuda.get_device_name(dev)}")
    else:
        print("CUDA not available")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    return device

In [5]:
device = get_device()

CUDA not available
Device: cpu


## Parameters

In [6]:
SEED = 17

VERSION = 0

NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
EPOCHS = 30          # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LAMBDA = 1e-4

torch.manual_seed(SEED)

## Dataset

In [8]:
# Define the Dataset class
class PACSDataset(Dataset):
    def __init__(self, domain, transform, root_dir):
        assert domain in ['photo', 'art_painting', 'cartoon', 'sketch']
        self.examples = [] # (img_path, class_label)
        self.T = transform

        with open(f'{root_dir}/PACS/{domain}.txt', 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.strip().split()
            img_path = f"{root_dir}/PACS/{line[0]}"
            class_label = int(line[1])
            self.examples.append((img_path, class_label))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, index):
        img_path, class_label = self.examples[index]
        img = Image.open(img_path).convert('RGB')
        img = self.T(img)
        return img, class_label

## Dataset preprocessing

In [9]:
def dataset_preprocessing():
    dataset_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Define the Dataset object for training & testing
    traindataset = PACSDataset(domain='cartoon', transform=dataset_transform, root_dir='./data')
    testdataset = PACSDataset(domain='sketch', transform=dataset_transform, root_dir='./data')

    # Define the DataLoaders
    trainloader = DataLoader(traindataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
    testloader = DataLoader(testdataset, batch_size=BATCH_SIZE, num_workers=4)
    valloader = testloader

    return trainloader, valloader, testloader

In [10]:
trainloader, valloader, testloader = dataset_preprocessing()

## Model definition

In [ ]:
def get_model():
    model = AlexNet()
    model.load_state_dict(AlexNet_Weights.IMAGENET1K_V1.get_state_dict(progress=True), strict=False)
    model.classifier[-1] = nn.Linear(4096, NUM_CLASSES)
    model = model.to(device)
    return model

def save_model(model, file_name):
    torch.save(model.state_dict(), file_name)

def load_model(model, file_name, device="cuda"):
    model.load_state_dict(torch.load(file_name, map_location=torch.device(device)))
    return model

## Loss function definition

In [ ]:
def get_loss_function():
    return nn.CrossEntropyLoss()

## Optimizer definition

In [ ]:
def get_optimizer(model):
    return torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

## Scheduler definition

In [ ]:
def get_scheduler(optimizer):
    return optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

## Plot

In [ ]:
def plot_training_metrics(
    train_losses, val_losses, train_accuracies, val_accuracies, learning_rates, model_number, base_dir
):
    fig = plt.figure()
    plt.title("Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.legend()
    plt.savefig(f"{base_dir}/plots/loss_{model_number}.pdf")
    plt.close(fig)

    fig = plt.figure()
    plt.title("Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.plot(train_accuracies, label="Train Accuracy")
    plt.plot(val_accuracies, label="Val Accuracy")
    plt.legend()
    plt.savefig(f"{base_dir}/plots/accuracy_{model_number}.pdf")
    plt.close(fig)

    fig = plt.figure()
    plt.title("Learning rate")
    plt.ylabel("learning rate")
    plt.xlabel("Epoch")
    plt.plot(learning_rates, label="Learning Rate")
    plt.legend()
    plt.savefig(f"{base_dir}/plots/learning_rate_{model_number}.pdf")
    plt.close(fig)

## Training

In [ ]:
def train_baseline(model, model_number, trainloader, valloader, loss_function, optimizer, scheduler, device, monitor, base_dir):
    cudnn.benchmark = True

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    learning_rates = []

    best_val_loss = None

    for e in range(EPOCHS):
        monitor.start(desc=f"Epoch {e + 1}/{EPOCHS}", max_progress=len(trainloader))

        learning_rate = scheduler.get_last_lr()[0]
        learning_rates.append(learning_rate)

        train_loss = 0.0
        cumulative_loss = 0.0
        count_loss = 0

        train_accuracy = 0.0
        correct_predictions = 0
        count_predictions = 0

        model.train()
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logits, _ = model(inputs)

            loss = loss_function(logits, labels)

            cumulative_loss += loss.item()
            count_loss += 1

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss = cumulative_loss / count_loss

            predicted_labels = torch.argmax(logits, dim=1)
            count_predictions += labels.size(0)
            correct_predictions += (predicted_labels == labels).sum().item()
            train_accuracy = correct_predictions / count_predictions

            monitor.update(
                i + 1,
                learning_rate=f"{learning_rate:.5f}",
                train_loss=f"{train_loss:.4f}",
                train_accuracy=f"{train_accuracy:.4f}",
            )

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        monitor.stop()

        if valloader is not None:
            monitor.start(desc=f"Validation", max_progress=len(valloader))

            val_loss = 0.0
            cumulative_loss = 0.0
            count_loss = 0

            val_accuracy = 0.0
            correct_predictions = 0
            count_predictions = 0

            model.eval()
            with torch.no_grad():
                for i, (inputs, labels) in enumerate(valloader):
                    inputs, labels = inputs.to(device), labels.to(device)
                    logits, _ = model(inputs)
                    loss = loss_function(logits, labels)
                    cumulative_loss += loss.item()
                    count_loss += 1
                    val_loss = cumulative_loss / count_loss
                    predicted_labels = torch.argmax(logits, dim=1)
                    count_predictions += labels.size(0)
                    correct_predictions += (predicted_labels == labels).sum().item()
                    val_accuracy = correct_predictions / count_predictions
                    monitor.update(
                        i + 1,
                        val_loss=f"{val_loss:.4f}",
                        val_accuracy=f"{val_accuracy:.4f}",
                    )

            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)
            monitor.stop()

            if best_val_loss is None or val_loss < best_val_loss:
                save_model(model, f"{base_dir}/weights/best_{model_number}.pt")
                monitor.log(f"Model saved as best_{model_number}.pt\n")
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1

        scheduler.step()

        save_model(model, f"{base_dir}/weights/last_{model_number}.pt")

        plot_training_metrics(
            train_losses,
            val_losses,
            train_accuracies,
            val_accuracies,
            learning_rates,
            model_number,
            base_dir,
        )

    monitor.print_stats()

In [ ]:
def train_dann(model, model_number, trainloader, valloader, loss_function, optimizer, scheduler, device, monitor, base_dir):
    cudnn.benchmark = True

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    learning_rates = []

    best_val_loss = None

    for e in range(EPOCHS):
        monitor.start(desc=f"Epoch {e + 1}/{EPOCHS}", max_progress=len(trainloader))

        learning_rate = scheduler.get_last_lr()[0]
        learning_rates.append(learning_rate)

        train_loss = 0.0
        cumulative_loss = 0.0
        count_loss = 0

        train_accuracy = 0.0
        correct_predictions = 0
        count_predictions = 0

        model.train()
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            logits, _ = model(inputs)

            loss = loss_function(logits, labels)

            cumulative_loss += loss.item()
            count_loss += 1

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss = cumulative_loss / count_loss

            predicted_labels = torch.argmax(logits, dim=1)
            count_predictions += labels.size(0)
            correct_predictions += (predicted_labels == labels).sum().item()
            train_accuracy = correct_predictions / count_predictions

            monitor.update(
                i + 1,
                learning_rate=f"{learning_rate:.5f}",
                train_loss=f"{train_loss:.4f}",
                train_accuracy=f"{train_accuracy:.4f}",
            )

        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        monitor.stop()

        if valloader is not None:
            monitor.start(desc=f"Validation", max_progress=len(valloader))

            val_loss = 0.0
            cumulative_loss = 0.0
            count_loss = 0

            val_accuracy = 0.0
            correct_predictions = 0
            count_predictions = 0

            model.eval()
            with torch.no_grad():
                for i, (inputs, labels) in enumerate(valloader):
                    inputs, labels = inputs.to(device), labels.to(device)
                    logits, _ = model(inputs)
                    loss = loss_function(logits, labels)
                    cumulative_loss += loss.item()
                    count_loss += 1
                    val_loss = cumulative_loss / count_loss
                    predicted_labels = torch.argmax(logits, dim=1)
                    count_predictions += labels.size(0)
                    correct_predictions += (predicted_labels == labels).sum().item()
                    val_accuracy = correct_predictions / count_predictions
                    monitor.update(
                        i + 1,
                        val_loss=f"{val_loss:.4f}",
                        val_accuracy=f"{val_accuracy:.4f}",
                    )

            val_losses.append(val_loss)
            val_accuracies.append(val_accuracy)
            monitor.stop()

            if best_val_loss is None or val_loss < best_val_loss:
                save_model(model, f"{base_dir}/weights/best_{model_number}.pt")
                monitor.log(f"Model saved as best_{model_number}.pt\n")
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1

        scheduler.step()

        save_model(model, f"{base_dir}/weights/last_{model_number}.pt")

        plot_training_metrics(
            train_losses,
            val_losses,
            train_accuracies,
            val_accuracies,
            learning_rates,
            model_number,
            base_dir,
        )

    monitor.print_stats()

In [ ]:
#### TRAINING LOOP
model.train()

# DANN

for epoch in range(NUM_EPOCHS):
    epoch_loss = [0.0, 0]
    for batch_idx, ((src_x, src_y), (trg_x, _)) in tqdm(enumerate(zip(train_loader, test_loader))):
        src_x, src_y = src_x.to(device), src_y.to(device)
        trg_x = trg_x.to(device)

        src_cls_o, src_dom_o = model(src_x)
        _, trg_dom_o = model(trg_x)

        if batch_idx % 2 == 0:
            # Classification Loss
            loss = F.cross_entropy(src_cls_o, src_y)

        else:
            # Classification Loss
            cls_loss = F.cross_entropy(src_cls_o, src_y)

            # Source Domain Adversarial Loss --> src_dom_label = 0
            src_dom_label = torch.zeros(src_dom_o.size(0)).long().to(device)
            src_dom_loss = F.cross_entropy(src_dom_o, src_dom_label)

            # Target Domain Adversarial Loss --> trg_dom_label = 1
            trg_dom_label = torch.ones(trg_dom_o.size(0)).long().to(device)
            trg_dom_loss = F.cross_entropy(trg_dom_o, trg_dom_label)

            # Final Loss
            loss = cls_loss - LAMBDA * (src_dom_loss + trg_dom_loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss[0] += loss.item()
        epoch_loss[1] += src_x.size(0)

    scheduler.step()
    print(f'[EPOCH {epoch+1}] Avg. Loss: {epoch_loss[0] / epoch_loss[1]}')
# pip install wandb

#### TEST LOOP
model.eval()

meter = Accuracy(task='multiclass', num_classes=NUM_CLASSES).to(device)

with torch.no_grad():
    for x, y in tqdm(test_loader):
        x, y = x.to(device), y.to(device)
        cls_o, _ = model(x)
        meter.update(cls_o, y)
accuracy = meter.compute()

print(f'\nAccuracy on the target domain: {100 * accuracy:.2f}%')

NameError: name 'model' is not defined